In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [16]:
data = pd.read_csv(r"C:\Users\furka\Desktop\ain4311\project\real-time-motion-detector\arkadas.csv")

In [17]:
# Özellikler ve etiketleri ayırma
features = data[['mean', 'top_3_mean', 'min', 'max', 'std_dev', 'median', 'q1', 'q3', 'skewness', 'dominant_freq', 'energy']]
labels = data['activity']

# Etiketleri encode etme
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Özellikleri normalize etme
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Veriyi zaman adımlarına bölme
def create_sequences(data, labels, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(labels[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 10
X, y = create_sequences(features_scaled, labels_encoded, time_steps)

# Veriyi eğitim ve test setlerine bölme
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
# Özellikler ve etiketleri ayırma
features = data[['mean', 'top_3_mean', 'min', 'max', 'std_dev', 'median', 'q1', 'q3', 'skewness', 'dominant_freq', 'energy']]
labels = data['name']  # "name" sütununu hedef değişken olarak seçiyoruz

# Etiketleri encode etme
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)  # İsimleri sayısal değerlere dönüştür

# Özellikleri normalize etme
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Veriyi zaman adımlarına bölme
def create_sequences(data, labels, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(labels[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 10
X, y = create_sequences(features_scaled, labels_encoded, time_steps)

# Veriyi eğitim ve test setlerine bölme
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# LSTM modeli oluşturma
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='softmax'))  # Sınıf sayısı kadar nöron ve softmax aktivasyonu

# Modeli derleme
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50


C:\Users\furka\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.3758 - loss: 1.0516 - val_accuracy: 0.5654 - val_loss: 0.9371
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5323 - loss: 0.9482 - val_accuracy: 0.5561 - val_loss: 0.8769
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5210 - loss: 0.9075 - val_accuracy: 0.5981 - val_loss: 0.8329
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5268 - loss: 0.8867 - val_accuracy: 0.6075 - val_loss: 0.8639
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5579 - loss: 0.8624 - val_accuracy: 0.5841 - val_loss: 0.7571
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5578 - loss: 0.8337 - val_accuracy: 0.6355 - val_loss: 0.7951
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6349 - loss: 0.7423 - val_accuracy: 0.6822 - val_loss: 0.7333
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6687 - loss: 0.7185 - val_accuracy: 0.7009 - val_loss: 0.

In [20]:
from sklearn.metrics import classification_report, confusion_matrix

# Modeli değerlendirme
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Doğruluk Skoru: {test_accuracy}")

# Tahminler ve sınıflandırma raporu
y_pred = np.argmax(model.predict(X_test), axis=1)
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("Karışıklık Matrisi:\n", confusion_matrix(y_test, y_pred))


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9275 - loss: 0.1858
Test Doğruluk Skoru: 0.9299065470695496
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Sınıflandırma Raporu:
               precision    recall  f1-score   support

        Enes       0.98      0.80      0.88        65
      Furkan       0.94      0.99      0.96        77
       Yusuf       0.89      0.99      0.93        72

    accuracy                           0.93       214
   macro avg       0.94      0.92      0.93       214
weighted avg       0.93      0.93      0.93       214

Karışıklık Matrisi:
 [[52  5  8]
 [ 0 76  1]
 [ 1  0 71]]


In [21]:
# Yeni veri için tahmin yapma
new_data = features_scaled[:time_steps]  # Örnek olarak ilk 10 adımı aldık
new_data = np.expand_dims(new_data, axis=0)  # Model girişine uygun hale getirme
prediction = model.predict(new_data)
predicted_name = label_encoder.inverse_transform([np.argmax(prediction)])
print(f"Tahmin edilen isim: {predicted_name[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Tahmin edilen isim: Yusuf


In [22]:
train_loss, train_accuracy = model.evaluate(X_train, y_train)
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Eğitim doğruluğu: {train_accuracy:.4f}")
print(f"Test doğruluğu: {test_accuracy:.4f}")


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8971 - loss: 0.2412
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9275 - loss: 0.1858
Eğitim doğruluğu: 0.8960
Test doğruluğu: 0.9299
